In [1]:
import os
import requests
import json
import uuid
import pandas as pd
from icecream import ic

In [2]:
api_key = os.environ["GOOGLE_MAPS_API_KEY"]

List Place Type: https://developers.google.com/maps/documentation/places/web-service/supported_types?hl=id

In [3]:
place_types = '''accounting
airport
amusement_park
aquarium
art_gallery
atm
bakery
bank
bar
beauty_salon
bicycle_store
book_store
bowling_alley
bus_station
cafe
campground
car_dealer
car_rental
car_repair
car_wash
casino
cemetery
church
city_hall
clothing_store
convenience_store
courthouse
dentist
department_store
doctor
drugstore
electrician
electronics_store
embassy
fire_station
florist
funeral_home
furniture_store
gas_station
gym
hair_care
hardware_store
hindu_temple
home_goods_store
hospital
insurance_agency
jewelry_store
laundry
lawyer
library
light_rail_station
liquor_store
local_government_office
locksmith
lodging
meal_delivery
meal_takeaway
mosque
movie_rental
movie_theater
moving_company
museum
night_club
painter
park
parking
pet_store
pharmacy
physiotherapist
plumber
police
post_office
primary_school
real_estate_agency
restaurant
roofing_contractor
rv_park
school
secondary_school
shoe_store
shopping_mall
spa
stadium
storage
store
subway_station
supermarket
synagogue
taxi_stand
tourist_attraction
train_station
transit_station
travel_agency
university
veterinary_care
zoo'''

list_place_types = place_types.split('\n')

In [4]:
input_latitude = '-6.89742292289528'

In [5]:
input_longitude = '109.678618210094'

In [6]:
input_radius = 600

In [7]:
input_place_type = 'cafe'

Utilize the data from dummy.json first to avoid excessive requests to Google Maps.

In [8]:
def near_by_search(api_key=None, latitude=None, longitude=None, radius=None, place_type=None, business_status='operational', dummy=False):
    if dummy:
        file_path = "dummy.json"
        with open(file_path, "r") as f:
            data = json.load(f)
    else:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&type={place_type}&business_status={business_status}&key={api_key}"
        response = requests.get(url)
        data = json.loads(response.text)
    
    return data

In [9]:
def convert_businesses_into_df(api_key=None, results=None, place_id_for_debug=None):
    businesses = []
    for result in results['results']:
        id = str(uuid.uuid4())
        name = result["name"]
        address = result["vicinity"]
        latitude = result["geometry"]["location"]["lat"]
        longitude = result["geometry"]["location"]["lng"]
        rating = result.get("rating", 0)
        compound_code = result.get("plus_code", {}).get("compound_code", "N/A")
        categories = ", ".join(result.get("types", []))
        photo = result["photos"][0]["photo_reference"] if "photos" in result else "N/A"
        place_id = result["place_id"]

        # more details
        file_path = f'business_details/{result["place_id"]}.json'
        if os.path.exists(file_path):
            with open(file_path, "r") as f:
                details_data = json.load(f)
        else:
            details_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={result["place_id"]}&key={api_key}'
            details_response = requests.get(details_url)
            details_data = json.loads(details_response.text)
            
            with open(file_path, "w") as f:
                json.dump(data, f)

        for component in details_data["result"]["address_components"]:
            if "administrative_area_level_1" in component["types"]:
                province = component["long_name"]
            elif "administrative_area_level_2" in component["types"]:
                regency = component["long_name"]
            elif "administrative_area_level_3" in component["types"]:
                district = component["long_name"]
            elif "administrative_area_level_4" in component["types"]:
                village = component["long_name"]
            elif "postal_code" in component["types"]:
                postal_code = component["long_name"]

        url = details_data["result"]["url"]

        if place_id_for_debug and place_id == place_id_for_debug:
            ic(details_data["result"])

        businesses.append(
            {
                "id": id,
                "place_id": place_id,
                "categories": categories,
                "province": province,
                "regency": regency,
                "district": district,
                "village": village,
                "postal_code": postal_code,
                "name": name,
                "latitude": latitude,
                "longitude": longitude,
                "photo": photo,
                "rating": rating,
                "url": url,
            }
        )

    columns = list(businesses[0].keys())
    return pd.DataFrame(businesses, columns=columns)

In [10]:
results = near_by_search(api_key, dummy=True)

In [11]:
df = convert_businesses_into_df(api_key, results)

In [12]:
df_businesses = _deepnote_execute_sql('SELECT * FROM df', 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_businesses

,id,place_id,categories,province,regency,district,village,postal_code,name,latitude,longitude,photo,rating,url
0,e3758b56-021e-4641-9a0a-044be4620022,ChIJa6drWDokcC4RwtALTn71ixk,"cafe, food, point_of_interest, establishment",Central Java,Pekalongan City,Pekalongan Timur,Noyontaan,51128,Teh Tong TJI,-6.897833,109.678888,N/A,5.0,https://maps.google.com/?cid=1840834795535061186
1,6ecc5a76-e26b-40fa-bbf7-167c9bf3083c,ChIJxVCUXCclcC4RNHd7B8RYVhA,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Keputran,51128,Cava milkshake,-6.897344,109.678499,N/A,0.0,https://maps.google.com/?cid=1177225951566329652
2,a1e578a4-e9f8-4909-94a9-e182a64286d1,ChIJrXKnU6glcC4R40dDi2AiGVc,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Keputran,51128,Angkringan gondrong,-6.897316,109.678493,N/A,0.0,https://maps.google.com/?cid=6276085353766143971
3,e73d644d-49e4-4f5e-8f00-70b3a415bf40,ChIJ55voFBolcC4RcoFS4YZs-nA,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Keputran,51128,Angkringan Memet,-6.897316,109.678493,AZose0n_KskzcWAtJHjS9IVkkOVyhNFdFe0l6Z4YvUfpGd...,5.0,https://maps.google.com/?cid=8140938602987290994
4,ab2bacef-ca3e-457b-a7e6-873ca22c60ab,ChIJrX1IizokcC4RlOXRvabIipY,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Keputran,51128,Seblak food sundanese,-6.897264,109.678521,AZose0lbz4jXJE-r4uUesDSgP7AoDEXVShhpuvfo2SMW_e...,4.0,https://maps.google.com/?cid=10847703270950036884
5,6190c176-da98-4591-bd3b-a70aca955948,ChIJqa_VGSslcC4RJsDiEhyV7EE,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Keputran,51128,brabuuz,-6.897159,109.678269,N/A,0.0,https://maps.google.com/?cid=4750335654777503782
6,4a24ffdd-fe10-427f-9f3c-167a9c5c3a28,ChIJJRRQ1JglcC4Rt15FbkT-dpk,"cafe, food, point_of_interest, establishment",Central Java,Pekalongan City,Pekalongan Timur,Noyontaan,51128,HOBI4D cafe,-6.897801,109.679060,N/A,0.0,https://maps.google.com/?cid=11058305504916102839
7,b13bcd71-4b86-421e-9db2-7a44603109a9,ChIJRwf4WzokcC4RoIVK-wFjZNM,"cafe, food, point_of_interest, establishment",Central Java,Pekalongan City,Pekalongan Timur,Noyontaan,51129,Pak Herman Coffee and Guyon,-6.897581,109.679227,AZose0noul_ivG6M36oH_nTj9jq45g-qvJH746ZXBIX4X0...,4.7,https://maps.google.com/?cid=15232408699835942304
8,7c95570a-bf5d-4ad0-b647-7319ab693584,ChIJRR6sqjskcC4RxIsAa3Uxag8,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Noyontaan,51128,A&S Coffee,-6.898061,109.678906,AZose0la7-t2JYADmt5Np9BfcynnBzG9Z07Jm47SV3zH3B...,5.0,https://maps.google.com/?cid=1110754638476381124
9,f93981c4-1d47-4af5-a41c-d20692b7c0d0,ChIJq_QjWTokcC4Rvu4VG3OfX_w,"cafe, food, point_of_interest, establishment",Jawa Tengah,Kota Pekalongan,Kecamatan Pekalongan Timur,Noyontaan,51128,Mata cafe,-6.898050,109.678957,AZose0md1RwkXH2QjsQwR821kh8DfvwBYZ7ZzYQxKY-79V...,4.2,https://maps.google.com/?cid=18185429137069829822


In [13]:
df.to_csv("google_maps_data.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa0ff5c2-b244-406d-b1f4-a2f4cd653e27' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>